# Amazon data set analysis

With the given data (amazon-meta.txt.gz) for 548552 products, perform exploratory analysis and make suggestions for further analysis on the following aspects.

## 1. Trustworthiness of ratings

Ratings are susceptible to manipulation, bias etc. What can you say (quantitatively speaking) about the ratings in this dataset?

In order to answer this question, let's focus on the following data:
  - average rating
  - number of downloads
  - votes for a rating
  - rating helpfulness

We are interested in a product that has more than 10 ratings, at least 5 rating votes and 5 rating helpfulness votes.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import numpy as np
from scipy.stats.stats import pearsonr

In [2]:
import gzip

row_limit = -1
row_count = 0

# create the empty dataframe
dataDF = pd.DataFrame(columns = ['id', 'asin', 'group', 'salesrank', 'categories'])

# the collection we use to fill the dataframe
collection = []

# the collection we use to load the rating for each of the product
ratings_tab = []

# the regular expression for review header
review_reg = 'reviews: total:.*downloaded:.*avg rating:.*'

# regular expression for review item lines
rev_item_reg = '\d{4}-\d+-\d+  cutomer: .*  rating: \d  votes:   \d+  helpful:   \d+'

with gzip.open('amazon-meta.txt.gz') as in_file:
    for line in in_file: 
        line_s = line.strip().decode('utf-8').lower()
        if len(line_s) == 0  or line_s[0] == '#':
            continue
        row_count += 1
        rating_item = {}
        
        # if a product id is encountered we store the collected information and clean the temporary storage vars
        if re.match('id:\s+\d+', line_s) is not None:
            if len(ratings_tab) > 9:
                ratings_array = [float(x['rating']) for x in ratings_tab]
                votes_array = [float(x['votes']) for x in ratings_tab]
                h_array = [float(x['helpful']) for x in ratings_tab]
                rec['rating_sd'] = np.std(ratings_array)
                rec['votes_avg'] = np.mean(votes_array)
                rec['votes_sd'] = np.std(votes_array)
                rec['help_avg'] = np.mean(h_array)
                rec['help_sd'] = np.std(h_array)
                rec['hv_corr'] = pearsonr(h_array, 
                                             y=votes_array)[0]
                rec['help_fratio'] = sum([ (1 if x > 0 else 0) for x in h_array])/len(ratings_tab)
                rec['votes_fratio'] = sum([ (1 if x > 0 else 0) for x in votes_array])/len(ratings_tab)
                collection.append(rec)
            rec = {}
            ratings_tab = []
            rec['id'] = line_s.split(':')[1].strip()
            
        
        if 'asin:' in line_s:
            rec['asin'] = line_s.split(':')[1].strip()
        if 'salesrank:' in line_s:
            rec['salesrank'] = line_s.split(':')[1].strip()
        if 'group:' in line_s:
            rec['group'] = line_s.split(':')[1].strip()
        if 'categories:' in line_s:
            rec['categories'] = line_s.split(':')[1].strip()
        if re.match(review_reg, line_s) is not None:
            rec['reviews_average'] = line_s.split(':')[4].strip()
            rec['reviews_total'] = re.findall('\d+', line_s.split(':')[2])[0]
            rec['reviews_downloads'] = re.findall('\d+', line_s.split(':')[2])[0]
        if re.match(rev_item_reg, line_s):
            rating_item['rating'] = re.findall('\d+', line_s.split(':')[2])[0]
            rating_item['votes'] = re.findall('\d+', line_s.split(':')[3])[0]
            rating_item['helpful'] = re.findall('\d+', line_s.split(':')[4])[0]
            ratings_tab.append(rating_item)
            
        if row_count == row_limit:
            break
# fill the dataframe with results
dataDF = dataDF.append(collection)

# delete the collection variable
del collection

/Users/klitonandrea/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [3]:
dataDF.head()

,asin,categories,group,help_avg,help_fratio,help_sd,hv_corr,id,rating_sd,reviews_average,reviews_downloads,reviews_total,salesrank,votes_avg,votes_fratio,votes_sd
0,0738700797,2,book,5.100000,1.000000,1.920937,0.896439,2,0.458258,4.5,12,12,168596,5.500,1.000000,2.061553
1,1590770218,9,book,3.900000,1.000000,1.757840,0.631842,12,1.577973,4.5,12,12,24741,5.600,1.000000,1.854724
2,0195110382,3,book,2.400000,0.900000,1.280625,0.925996,16,0.670820,4.5,10,10,631564,3.000,1.000000,1.264911
3,b000007r0t,1,music,0.928571,0.357143,1.667517,0.553947,18,0.515079,5,15,15,109301,2.500,0.571429,2.745126
4,0790747324,26,dvd,2.200000,0.848000,1.943193,0.839643,21,0.724254,4.5,140,140,795,3.072,0.904000,2.280968


We convert to numeric those columns that are numeric but of text type:

In [4]:
dataDF.reviews_average = pd.to_numeric(dataDF.reviews_average)
dataDF.reviews_downloads =  pd.to_numeric(dataDF.reviews_downloads) 
dataDF.reviews_total = pd.to_numeric(dataDF.reviews_total)

Now, we can have a look at the features that were extracted:

In [5]:
dataDF.describe()

,help_avg,help_fratio,help_sd,hv_corr,rating_sd,reviews_average,reviews_downloads,reviews_total,votes_avg,votes_fratio,votes_sd
count,94091.000000,94091.000000,94091.000000,94039.000000,94091.000000,94091.000000,94091.000000,94091.000000,94091.000000,94091.000000,94091.000000
mean,2.070926,0.711468,1.817849,0.827339,0.881573,4.228656,67.647756,67.647756,2.917819,0.805622,2.158023
std,1.006368,0.195810,0.505384,0.138606,0.398520,0.573216,163.129870,163.129870,1.246073,0.172554,0.491143
min,0.000000,0.000000,0.000000,-0.575055,0.000000,1.000000,10.000000,10.000000,0.000000,0.000000,0.000000
25%,1.312500,0.583333,1.481741,0.764384,0.594816,4.000000,16.000000,16.000000,2.000000,0.708333,1.880742
50%,1.942857,0.736842,1.833030,0.854374,0.881396,4.500000,27.000000,27.000000,2.815217,0.846154,2.215789
75%,2.705882,0.866667,2.165641,0.927713,1.183216,4.500000,57.000000,57.000000,3.750000,0.936508,2.490799
max,7.454545,1.000000,3.924283,1.000000,2.000000,5.000000,5545.000000,5545.000000,8.000000,1.000000,4.049984


From the table above we do see that 'reviews_total' is duplicated by 'reviews_downloads' or at least they are strongly correlated. 

Another feature of interest is 'hv_corr'. It captures the correlation between the votes that a review has received and the helpfulness of the review. A negative correlation denotes the reviews that are highly voted but not helpful or vice versa. We will consider trustworthy, those reviews that have a strong positive correlation coefficient given by 'hv_corr'.

## 2. Category bloat

Consider the product group named 'Books'. Each product in this group is associated with categories. Naturally, with categorization, there are tradeoffs between how broad or specific the categories must be.

For this dataset, quantify the following:

    a. Is there redundancy in the categorization? How can it be identified/removed?
    b. Is it possible to reduce the number of categories drastically (say to 10% of existing categories) by sacrificing relatively few category entries (say close to 10%)?

To answer the questions above we need to count all the categories for 'Books' group that are listed for each item.

In [6]:
row_limit = -1
row_count = 0

# dictionary that holds all category counts
categories_count = {}

# dictionary that holds all leaf category counts
categories_entr_count = {}

# the collection we use to load the rating for each of the product
categories_lines = []

# the regular expression for category item
cat_item_reg = "(\|.*\[.*\])+"

with gzip.open('amazon-meta.txt.gz') as in_file:
    for line in in_file: 
        line_s = line.strip().decode('utf-8').lower()
        if len(line_s) == 0  or line_s[0] == '#':
            continue
        row_count += 1
        # if a product id is encountered we store the collected information and clean the temporary storage vars
        if re.match('id:\s+\d+', line_s) is not None:
            
            if 'group' not in rec:
                continue
            if rec['group'] != 'book':
                rec = {}
                categories_lines = []
                continue
            for cline in categories_lines:
                cats = cline.split('|')
                
                # considering only the last category (leaf) in category entry
                category = cats[len(cats) - 1]
                if category in categories_entr_count:
                    categories_entr_count[category] += 1
                else:
                    categories_entr_count[category] = 1
                
                # considering all categories in category entry
                for category in cats:
                    if category in categories_count:
                        categories_count[category] += 1
                    else:
                        categories_count[category] = 1
            rec={}                
            categories_lines = []
            
        if 'group:' in line_s:
            rec['group'] = line_s.split(':')[1].strip()
        if re.match(cat_item_reg, line_s) is not None:
            categories_lines.append(line_s)
        if row_count == row_limit:
            break  

In [7]:
import operator
sorted_categories_by_key = sorted(categories_count.items(), key=operator.itemgetter(0))
sorted_categories_by_val = sorted(categories_count.items(), key = operator.itemgetter(1), reverse=True)

In [8]:
sum_of_sorted = sum([x[1] for x in sorted_categories_by_val[1:]])

In [9]:
sorted_categories_by_key[1:]

[('( a )[12833]', 78),
 ('( a )[14653]', 398),
 ('( a )[170541]', 541),
 ('( a )[287121]', 53),
 ('( a )[287148]', 7),
 ('( a )[297872]', 203),
 ('( a )[5440]', 72),
 ('( a )[585708]', 10),
 ('( a )[69301]', 31),
 ('( a )[70022]', 629),
 ('( a )[70740]', 8),
 ('( a )[70836]', 33),
 ('( a )[912594]', 113),
 ('( a )[916930]', 95),
 ('( a-c )[1105]', 281),
 ('( b )[12855]', 290),
 ('( b )[14711]', 762),
 ('( b )[170542]', 1539),
 ('( b )[287122]', 45),
 ('( b )[287149]', 11),
 ('( b )[297873]', 390),
 ('( b )[411316]', 4),
 ('( b )[5603]', 138),
 ('( b )[585600]', 43),
 ('( b )[585710]', 9),
 ('( b )[69302]', 344),
 ('( b )[70023]', 763),
 ('( b )[70741]', 29),
 ('( b )[70837]', 28),
 ('( b )[7346]', 13),
 ('( b )[912596]', 109),
 ('( b )[916932]', 135),
 ('( c )[12660]', 4),
 ('( c )[12894]', 277),
 ('( c )[14875]', 458),
 ('( c )[170543]', 790),
 ('( c )[287123]', 169),
 ('( c )[287150]', 67),
 ('( c )[297874]', 649),
 ('( c )[416120]', 1),
 ('( c )[585602]', 6),
 ('( c )[585712]', 4),


From the above we do see a lot of repetitive category names. The only difference between them is the category id inside []. The categories should get consolidated in order to reduce the redundancy.

A consolidated database of category entries would reduce the redundancy by:

In [11]:
len(set([x[0].split('[')[0] for x in sorted_categories_by_key[1:]]))/len(categories_count)

0.5744438488340927

In [12]:
len([x for x in sorted_categories_by_val if x[1] < 4])/len(set([x[0].split('[')[0] for x in sorted_categories_by_key[1:]]))

0.2948792721334422

Another approach is to consider only the last category item in the category entry (hierarchical chain of categories) that we have already retrieved in *categories_entr_count*:

In [14]:
len(categories_entr_count)

12853

In [17]:
categories_entr_count

{'preaching[12368]': 364,
 'sermons[12370]': 315,
 'wicca[12484]': 490,
 'witchcraft[12486]': 527,
 'general[5144]': 5108,
 'new testament[12159]': 604,
 'discipleship[12335]': 343,
 'life application[572080]': 90,
 'new testament[572082]': 1782,
 'general[572094]': 2067,
 'prayerbooks[12470]': 276,
 'business[297488]': 270,
 'photo essays[2082]': 2748,
 'general[4870]': 1983,
 'general[4993]': 451,
 'urban[11296]': 513,
 'photo essays[733676]': 2748,
 'general[10716]': 760,
 'criminology[11005]': 868,
 'general[11083]': 7715,
 'u.s.[11117]': 1253,
 'bread[4197]': 187,
 'general[10213]': 3093,
 'history & theory[11086]': 1633,
 'cultural[11235]': 2243,
 'general[4701]': 1404,
 'general[14046]': 508,
 'general[227584]': 389,
 'jp-unknown2[1061350]': 2833,
 'herbs, spices & condiments[4211]': 158,
 'general[4233]': 6528,
 'diabetic & sugar-free[4318]': 134,
 'low fat[4323]': 264,
 'general[4628]': 842,
 'low carbohydrate[4639]': 99,
 'low sugar[4642]': 51,
 'low-fat diet[282849]': 356,
 

Again, there are a lot of repetitive category entries. We'll regroup those together:

In [22]:
category_reduction = {}

for cat_key in categories_entr_count.keys():
    new_cat = cat_key.split('[')[0]
    if new_cat in category_reduction:
        category_reduction[new_cat] += categories_entr_count[cat_key]
    else:
        category_reduction[new_cat] = categories_entr_count[cat_key]

In [23]:
len(category_reduction)

7912

In [26]:
sorted_categ_red_by_val = sorted(category_reduction.items(), key = operator.itemgetter(1), reverse=True)

In [27]:
sorted_categ_red_by_val

[('general', 555368),
 ('fiction', 20574),
 ('contemporary', 11948),
 ('reference', 11657),
 ('management', 9533),
 ('history', 7615),
 ('classics', 6470),
 ('entrepreneurship', 6466),
 ('united states', 6153),
 ('literary', 5741),
 ('nonfiction', 5647),
 ('historical', 5506),
 ('photo essays', 5496),
 ('picture books', 5333),
 ('action & adventure', 5273),
 ('inspirational', 4905),
 ('software development', 4872),
 ('leadership', 4382),
 ('humorous', 4038),
 ('paperback', 3778),
 ('women', 3534),
 ('jp-unknown1', 3447),
 ('devotionals', 3421),
 ('guides', 3305),
 ('science fiction, fantasy, & magic', 3286),
 ('anthologies', 3180),
 ('history & criticism', 3113),
 ('finance & investing', 2997),
 ('state & local', 2977),
 ('new testament', 2950),
 ('board books', 2916),
 ('popular culture', 2908),
 ('jp-unknown2', 2833),
 ('20th century', 2777),
 ('human resources & personnel management', 2650),
 ('suspense', 2634),
 ('software design', 2418),
 ('english (all)', 2387),
 ('essays & trave

Let's calculate the sum of all category frequencies:

In [28]:
sum_sorted_red = sum([x[1] for x in sorted_categ_red_by_val])

In [29]:
sum_sorted_red

1440329

Next, we find the index of the array for the category items that are encountered less than 10% in the product:

In [33]:
(np.cumsum([x[1]/sum_sorted_red for x in sorted_categ_red_by_val]) < 0.9).argmin()


1313

In [38]:
sorted_categ_red_by_val[1313]

('horoscopes', 119)

We can reduce the number of categories by 10% by removing all the category entries with a frequency less than 118.

Now, we can answer both the questions asked above:

    a. there is a redundancy of categories, there are a lot of repetitive entities but differ only by IDs. It means there were entered several categories.
    b. we can reduce further the number of categories by removing the least used (< 118)

## Part B

    1. How would build the product categorization from scratch, using similar/co-purchased information?
    
    There are 2 aspects that cause category items duplication: entering several times the same entity (and different ID) and also creating categories that are not necessary (less frequently used). To prevent entering the same category twice we should check whether it exists and also make sure there is no duplication by using another foreign language. Or another alternative is to search for similar category in the database before creating a new category. In order to generate (create) a new category we should check whether 119 or more books share a distinct feature that does not exist as an entry in the database.
    
    2. Product thinking
    Now, put on your 'product thinking' hat.
    a. Is it a good idea to show users the categorization hierarchy for items? 
    
   **As we saw above, we can identify redundancy issues related to category entries by exposing the full categorization hierarchy to users. Which is a weakness in the product and probably leads to a less than a great user experience. However, this level of transparency is necessary. On premises that the categorization is achieved automatically, this level of transparency gives a user an opportunity to provide feedback whenever a discrepancy is encountered. Showing the categorization hierarchy for items demonstrates that we have assessed the quality of the product and it complies with the basic requirements.**
    
    b. Is it a good idea to show users similar/co-purchased items?
    
   **Showing users other similar/co-purchased items is a way to influence them to buy more products. While it might seem aggressive, if done properly it can result in a big outcome. It is important to show users relevant similar items which they most probably will purchase. This strategy should play very well in combination with recommender systems.**
    
    c. Is it a good idea to show users reviews and ratings for items?
    
   **It has been established that most people decide to purchase items based on the peers' reviews. Unfortunately, bad reviews could draw the users away while good reviews might bias the decisions and make it more difficult for new items in the market. Combined with reviews, the ratings are a powerful way to judge about a quality of the product. As we saw above, voting a review and its helpfulness leads to a better judgement on rating trusworthiness.**
    